# Challenge Prompting

Resolver los siguientes ejercicios dejando el codigo con su ejecucion.

Importar las librerias necesarias y **correr las celdas para visualizar el resultado en cada ejercicio**.

In [2]:
## bloque importacion de librerias

import json
import ipywidgets as widgets
from IPython.display import display, clear_output

In [3]:
## bloque variables de entorno
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

api_key = os.getenv("COHERE_API_KEY")
print(api_key)  # Verify the key is loaded


OJPX9QruSuFGPCFROfbofsm9Xg1XJPAQU45Pxcir


In [12]:
ej="""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""

In [4]:
## bloque conexion a Cohere
import cohere
co = cohere.ClientV2()
# alternativa:
# co = cohere.ClientV2(api_key)




## Ejercicio 1

Extraccion de entidades

Utilizar el LLM para extraer las siguientes entidades del texto medico.

- Paciente:
    - Nombre
    - Edad
- Fecha de admisión
- Síntomas
- Diagnóstico
- Tratamiento recomendado

**Aclaracion:** 

La salida tiene que ser un **string con formato de tipo json**, el cual se convertira en un diccionario de Python.

Si la linea de conversion en test da error el ejercicio no esta completo.

In [ ]:
# ejemplo 

# texto a analizar
"""La paciente, María González, de 45 años, fue admitida en el Hospital Central el 5 de agosto de 2023 debido a síntomas de fatiga crónica y dolores musculares./
Tras una serie de análisis, se diagnosticó fibromialgia. La doctora a cargo, Laura Ramírez, recomendó un tratamiento basado en fisioterapia y medicamentos analgésicos. /
La próxima consulta está programada para el 15 de septiembre."""


# respuesta del LLM
{
  "paciente": {
    "nombre": "María González",
    "edad": 45
  },
  "fecha_admision": "2023-08-05",
  "sintomas": [
    "fatiga crónica",
    "dolores musculares"
  ],
  "diagnostico": "fibromialgia",
  "tratamiento": [
    "fisioterapia",
    "medicamentos analgésicos"
  ]
}

In [ ]:
#####

In [14]:
text_to_analize = """Sofía López, de 28 años, ingresó al Hospital Infantil el 3 de abril de 2023 debido a fiebre alta y tos persistente./
Después de varias pruebas, se le diagnosticó neumonía. La pediatra responsable, Dra. Claudia Torres, indicó tratamiento con antibióticos y reposo./
La próxima evaluación será el 10 de abril."""

In [38]:

co = cohere.ClientV2()
response = co.chat(
    model= "command-r-plus-08-2024",
    messages=[{"role": "system", "content": "Sos un asistente capaza de interpretar entidades en un texto medico. Extrae el nombre y edad del paciente, la fecha de admision, los sintomas,"
                "el diagnotico y  el tratamiento recomendado. Los sintomas deben estar por separado. Si hay mas de un tratamiento debe estar por separado. La fecha transformarla a formato dd/mm/aaaa." 
                "Si alguna de estas entidades no se encuentran en el texto no inventes informacion.  Dar la respuesta en formato JSON"},
              {"role": "user", "content": f"{text_to_analize}"}],
)

llm_response=response.message.content[0].text
print(llm_response)



{
  "paciente": {
    "nombre": "Sofía López",
    "edad": "28 años"
  },
  "fecha_admision": "03/04/2023",
  "sintomas": [
    "Fiebre alta",
    "Tos persistente"
  ],
  "diagnostico": "Neumonía",
  "tratamiento": [
    "Antibióticos",
    "Reposo"
  ]
}


In [39]:
# test

final_result = json.loads(llm_response)
final_result

{'paciente': {'nombre': 'Sofía López', 'edad': '28 años'},
 'fecha_admision': '03/04/2023',
 'sintomas': ['Fiebre alta', 'Tos persistente'],
 'diagnostico': 'Neumonía',
 'tratamiento': ['Antibióticos', 'Reposo']}

## Ejercicio 2

Tenemos dos funciones en Python, una llamada *'add_contact'* y otra llamada *'get_information'*.

**Utilizar algun LLM que permita funtion calling** y desarrollar un codigo secuencial automatico que consiga:

Interpretar la consulta del usuario, identificar a que funcion llamar, luego llamarla (si es que aplica) y darle una respuesta final al usuario.  (usar function calling para esta solucion)

La entrada a dicho codigo es la consulta del usuario, a continuacion algunos ejemplos:

- "Agrega a Juan Pérez con el número 555-1234 y el correo juanperez@mail.com."
- "Guarda a Lucía Gómez en mis contactos. Su teléfono es 555-5678 y su email es lucia.gomez@gmail.com."
- "Cual es el Email de Juan Pérez.?"

Salidas esperadas de dichos ejemplos (variaran porque las genera el LLM):
-  El contacto fue anadido con exito
-  Se anadio el contacto
-  El email de juan perez es juanperez@mail.com

Link de ayuda: https://github.com/cohere-ai/notebooks/blob/main/notebooks/agents/Vanilla_Tool_Use_v2.ipynb

In [5]:
def add_contact(name, phone, email):
    """
    Agrega un contacto al diccionario.
    Parámetros:
        name (str): Nombre del contacto.
        phone (str): Número de teléfono del contacto.
        email (str): Correo electrónico del contacto.
    Retorna:
        str: Mensaje confirmando la adición del contacto.
    """
    contacts[name] = {'phone': phone, 'email': email}
    return "Contacto añadido con éxito."

def get_information(name):
    """
    Recupera la información de un contacto.
    Parámetros:
        name (str): Nombre del contacto.
    Retorna:
        dict/str: Información del contacto o un mensaje si no existe.
    """
    if name in contacts:
        return contacts[name]
    else:
        return "Contacto no encontrado."
    


In [89]:
contacts['Joaquin Lopez']

{'tel': 15456663258, 'email': 'Joacolocolopez@gmail.com'}

In [87]:
contacts = {
                        'Joaquin Lopez':{'tel': 15456663258, 'email': 'Joacolocolopez@gmail.com'},
                      'Flavio Oncativo':{'tel': 1545554178, 'email': 'FOncativo@hotmail.com'}
}

In [ ]:
# TIPS
# Probar primero generando una funcion y llamarla, luego anadir la otra
# Plantearlo paso por paso en distintas celdas, analizar las salidas y las entradas, como identificamos a que funcion llamar?
# luego automatizar dentro de una sola celda


# Lo importante es entregar hasta donde lleguen, sea una funcion, las dos pero sin poder hacer el flujo automatico, lo que puedan, siempre y cuando este
# claro lo que se quizo hacer con comentarios.

In [86]:
function_map={"add_contact":add_contact,
              "get_information": get_information}

tools = [
    {
        "type": "function",
        "function": {  
            "name": "add_contact",
            "description": "Add contact to dict and returns a message confirming the addition of the contact",
            "parameters": {  
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string", 
                        "description": "Name of new contact"
                    },
                    "phone": {
                        "type": "string",
                        "description": "phone number of new contact"
                    },
                    "email": {
                        "type": "string",
                        "description": "email of new contact"
                    }
                },
                "required": ["name", "phone", "email"]  #
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_information",
            "description": "get information about of a contact...",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of contact"
                    }
                },
                "required": ["name"]
            }
        }
    }
]

In [34]:
import cohere
co = cohere.ClientV2()

In [83]:
system_promt=f"""Sos un asistente capaza de interpretar los pedidos. Tenes a dispocicion estas tools: {tools}
a la funcion llamada get_information la vas a utilizar cuando te consulten el mail o numero de telefono de un contacto. La funcion llamafa add_contact la vas a utilizar cuando te pidan agregar un contacto al diccionario {contacts}"""

#messages=[{"role": "system", "content": f"{system_promt}"},
#              {"role": "user", "content": "cual es el email de joaquin lopez"}]

messages=[
              {"role": "user", "content": "cual es el email de joaquin lopez"}]

response = co.chat(
    model="command-a-03-2025",
    messages=messages,
    tools=tools, 
)

if response.message.tool_calls:
    for tc in response.message.tool_calls:
        tool_result = function_map[tc.function.name](
            **json.loads(tc.function.arguments)
        )
        tool_content = []
        for data in tool_result:
            tool_content.append(
                {
                    "type": "document",
                    "document": {"data": json.dumps(data)},
                }
            )
            # Optional: add an "id" field in the "document" object, otherwise IDs are auto-generated
        messages.append(
            {
                "role": "tool",
                "tool_call_id": tc.id,
                "content": tool_content,
            }
        )



In [84]:
if response.message.tool_calls:
    print("Tool plan:")
    print(response.message.tool_plan,"\n")
        
    print("Tool calls:")
    #print(response.message.tool_calls[0].function.name)
    for call in response.message.tool_calls[0]:
        print(call[1])
        #print(f"Tool name: {call.function.name} | Parameters: {call.function.parameters}")

Tool plan:
Utilizaré la herramienta 'get_information' para obtener el correo electrónico de Joaquín López. 

Tool calls:
get_information_ejwr4q7qgfjf
function
name='get_information' arguments='{"name":"joaquin lopez"}'


In [85]:
response = co.chat(
    model="command-a-03-2025", messages=messages, tools=tools
)
print(response.message.content[0].text)


BadRequestError: headers: {'access-control-expose-headers': 'X-Debug-Trace-ID', 'cache-control': 'no-cache, no-store, no-transform, must-revalidate, private, max-age=0', 'content-encoding': 'gzip', 'content-type': 'application/json', 'expires': 'Thu, 01 Jan 1970 00:00:00 GMT', 'pragma': 'no-cache', 'vary': 'Origin,Accept-Encoding', 'x-accel-expires': '0', 'x-debug-trace-id': 'a0bc42c2e63e0af24b281d4ffcc3334a', 'x-endpoint-monthly-call-limit': '1000', 'x-trial-endpoint-call-limit': '40', 'x-trial-endpoint-call-remaining': '38', 'date': 'Thu, 27 Nov 2025 22:51:44 GMT', 'x-envoy-upstream-service-time': '16', 'server': 'envoy', 'via': '1.1 google', 'alt-svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000', 'transfer-encoding': 'chunked'}, status_code: 400, body: {'id': '12818b07-e942-4cec-a967-d78b8b691114', 'message': "invalid tool message at messages[1]: tool call id 'get_information_ejwr4q7qgfjf' not found in previous tool calls"}

In [81]:
tool_results = []
for tc in response.message.tool_calls:
    tool_call = {"name": tc.name, "parameters": tc.parameters}
    tool_output = functions_map[tc.name](**tc.parameters)
    tool_results.append({"call": tool_call, "outputs": [tool_output]})

print("Tool results:") 
for result in tool_results:
    print(result)

TypeError: 'NoneType' object is not iterable

## Ejercicio 3

Crear una funcion llamada "history_answer", que toma como parametro de entrada una pregunta sobre un contexto dado y la salida es la respuesta final del proceso impulsado por un LLM.

Dada una historia, el usuario podra hacer preguntas sobre la misma y el LLM debe responder siguiendo los siguientes lineamientos:

REQUISITOS DE LA RESPUESTA
- las respuestas deben ser en base a la historia
- ante la misma pregunta siempre debe responder de la misma manera.
- que responda en solo una oracion.
- el idioma que responde debe ser el mismo que con el que se pregunta (ingles, espanol, portugues).
- que agregue emojis en la oracion que resuman el contenido de la misma.
- que responda siempre en tercera persona.
- si la pregunta no tiene relacion alguna con el contexto, la respuesta debe ser 'Lo siento no puedo ayudarte con eso'.
- Responder con 'Hakuna Matata!' al final de **todas** las respuestas (no importa idioma ni cantidad de tokens).

**Ayudin**: 
- No se limiten a usar 1 solo request al LLM, pueden dividirlo en partes para que por un lado se verifique el idioma, por otro lado se verifique si la pregunta tiene relacion con el contexto, etc

- Estructuren bien el prompt procurando separar instrucciones, contexto(historia) y pregunta del usuario.

- Recuerden usar el system message y user message.



In [ ]:
# ejemplo flojo de estructura de prompt
# prompt = f"Responde a la pregunta: {pregunta} de manera concisa y divertida en base a la siguiente historia: {historia}"

In [ ]:
def history_answer(pregunta):
    # your code here

    return respuesta_al_usuario

In [ ]:
historia = """En un pequeño feudo medieval, Thomas, un joven campesino de dieciséis años, trabajaba desde el amanecer en los campos de trigo del señor feudal. El sol apenas había salido cuando él ya había arado más de lo que sus manos podían soportar. La vida era dura, pero su familia dependía de la cosecha para pagar los impuestos y mantener su hogar de madera y paja.

Un día, el feudo fue sacudido por noticias de guerra. El rey había llamado a todos los hombres en edad de luchar. Thomas sabía que, al igual que otros jóvenes, no tenía elección. Cambió la hoz por una lanza rudimentaria y se unió a la milicia local. Sin entrenamiento, fue empujado a un campo de batalla embarrado, donde el acero resonaba y los gritos de los hombres llenaban el aire.

La batalla fue un caos. Thomas, con el corazón latiendo en su pecho como un tambor de guerra, apenas podía distinguir amigo de enemigo. Logró esquivar una espada, pero cayó al suelo, cubierto de lodo y sangre. Levantándose, vio cómo un compañero caía junto a él, sus ojos abiertos, vacíos.

Cuando la batalla terminó, el silencio era tan profundo como el vacío que sentía. Thomas regresó al feudo, diferente, marcado por la muerte y la violencia. Su madre lo recibió con lágrimas en los ojos, pero él, con la mirada fija en el horizonte, sabía que la inocencia había quedado atrás, enterrada en aquel campo de batalla. La paz del feudo ya no era la misma; él tampoco."""


pregunta = # insertar pregunta relacionada  o  no


# respuesta
print(history_answer(pregunta))

## Ejercicio 4

Crear un chatbot sencillo impulsado por un LLM. 

Dicho bot esta destinado a un usuario final y debe cumplir las siguientes **condiciones en sus respuestas**:

- Responder en no mas de 70 tokens.
- Responder de manera positiva, con un tono entusiasta.
- Responder con consejos útiles, como si fueras un tutor.

 
**Otras consideraciones**:

Respetar el formato de la interfaz provista por el ejercicio.

Ademas agregar al codigo propuesto un historial de conversaciones para que el bot pueda mantener el hilo de lo que se esta hablando. Para probar no usen mas de 3 conversaciones anidadas para no enviarle tantos tokens.

Dejar impreso en el notebook el historial de la conversacion.

In [3]:
# Crear widgets de entrada y salida
input_box = widgets.Text(placeholder='Escribe tu mensaje aquí...')
send_button = widgets.Button(description='Enviar')
output_box = widgets.Output()

# chat_history = []

# Función de respuesta simulada del chatbot
def chatbot_response(message):
    # Aquí puedes conectar tu modelo o lógica de chatbot real
    responses = {
        "hola": "¡Hola! ¿En qué puedo ayudarte?",
        "adiós": "¡Hasta luego!",
    }

    return responses.get(message.lower(), "Lo siento, no entiendo esa pregunta.")

# Función de manejo del botón
def on_send_button_clicked(b):
    with output_box:
        clear_output(wait=True)
        user_message = input_box.value
        if user_message.strip():
            print(f"Tú: {user_message}")
            response = chatbot_response(user_message)
            print(f"Chatbot: {response}")
        input_box.value = ''

# Asociar función al botón
send_button.on_click(on_send_button_clicked)

# Mostrar widgets
display(input_box, send_button, output_box)

Text(value='', placeholder='Escribe tu mensaje aquí...')

Button(description='Enviar', style=ButtonStyle())

Output()

In [ ]:
# print(conversation_history)

### RECOMENDACIONES GENERALES

No se confien probando con un par de respuestas y ya, hagan minimo 5 pruebas por ejercicio para asi tener mas chances de visualizar errores en la generacion del contenido.

Prueben combinar LLMs con programacion convencional para los casos que vean convenientes (decisiones if else, respuestas estaticas, etc)

Prueben con distintos modelos de Cohere, hay algunos optimizados para ciertas aplicaciones.